In [11]:
!pip install bs4
!pip install openpyxl
!pip install fake_useragent

In [3]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [4]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Config/92.2.2788.20'

In [9]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:

        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?' \
              + 'from={}&'.format(param_dict['from']) \
              + 'size={}&'.format(param_dict['size']) \
              + 'sort={}&'.format(param_dict['sort']) \
              + 'title_only={}&'.format(param_dict['title_only']) \
              + 'domain={}&'.format(param_dict['domain']) \
              + 'modified%2Cformat=yyyy-MM-dd&' \
              + 'type={}&'.format(param_dict['type']) * hasType \
              + 'bloc={}&'.format(param_dict['bloc']) * hasBloc \
              + 'modified%2Cfrom={}&'.format(param_dict['dateFrom']) \
              + 'modified%2Cto={}&'.format(param_dict['dateTo']) \
              + 'query={}'.format(param_dict['query'])

        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table

    def get_articles(self,
                     param_dict,
                     time_step=37,
                     save_every=5,
                     save_excel=True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from ' \
                  + param_copy['dateFrom'] + ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._get_search_table(param_copy)], ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [15]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "4" # экономика
dateFrom = '2021-04-01'
dateTo = "2022-04-04"

if use_parser == "LentaRu":
    param_dict = {'query'     : query, 
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material, 
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2021-04-01', 'dateTo': '2022-04-04', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '4', 'domain': '1'}


In [16]:
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 10,
                         save_every = 5, 
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2022-02-25 to 2022-03-07
Parsing articles from 2022-03-08 to 2022-03-18
Parsing articles from 2022-03-19 to 2022-03-29
Parsing articles from 2022-03-30 to 2022-04-04
Finish
14044


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1067960,https://lenta.ru/news/2021/04/01/sahar/,В российские магазины перестали поставлять сахар,1617236100,1617279786,1,1,0,0,4,[198],https://icdn.lenta.ru/images/2021/04/01/03/202...,1617236100,Фото: Виталий Аньков / РИА Новости Илья Соболе...,Заводы приостановили продажи продукта торговым...,Фото: Виталий Аньков / РИА Новости Илья ... им...
1,1067977,https://lenta.ru/news/2021/04/01/trillion/,Байден представил план по спасению экономики н...,1617246403,1617246403,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2021/04/01/06/202...,1617246403,Фото: Reuters Милана Микаилова Президент США Д...,"Президент США отметил, что это крупнейшее влож...",Фото: Reuters Милана Микаилова Президент ... э...
2,1067979,https://lenta.ru/news/2021/04/01/business/,Бизнес в России увеличил прибыль,1617248460,1617252223,1,1,0,0,4,[9],https://icdn.lenta.ru/images/2021/04/01/06/202...,1617248460,Фото: Евгений Биятов / РИА Новости Милана Мика...,В январе 2021 года обороты российских предприя...,"Фото: Евгений Биятов / РИА Новости ... больше,..."
3,1067984,https://lenta.ru/news/2021/04/01/kredit/,Россиянам назвали способ исправить плохую кред...,1617253029,1617253029,1,1,0,0,4,"[10, 11]",https://icdn.lenta.ru/images/2021/04/01/07/202...,1617253029,Фото: Владимир Трефилов / РИА Новости Илья Соб...,Для этого следует завести новую кредитную карту,Фото: Владимир Трефилов / РИА Новости ... испр...
4,1067989,https://lenta.ru/news/2021/04/01/finansist/,Финансист предостерег россиян от покупки подде...,1617255774,1617255775,1,1,0,0,4,[],https://icdn.lenta.ru/images/2021/04/01/08/202...,1617255774,Фото: Globallookpress.com Лика Миргородская Фи...,Он призвал конвертировать валюту безналичным с...,Фото: Globallookpress.com Лика ... россиян от ...


In [18]:
df = pd.concat([
    pd.read_excel('lenta2021_2022.xlsx'),
    pd.read_excel('lenta2022_2023.xlsx'),
    pd.read_excel('lenta2023_2024.xlsx')
], ignore_index=True)
df.head()

,Unnamed: 0,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,0,1067960,https://lenta.ru/news/2021/04/01/sahar/,В российские магазины перестали поставлять сахар,1617236100,1617279786,1,1,0,0,4,[198],https://icdn.lenta.ru/images/2021/04/01/03/202...,1617236100,Фото: Виталий Аньков / РИА Новости Илья Соболе...,Заводы приостановили продажи продукта торговым...,Фото: Виталий Аньков / РИА Новости Илья ... им...
1,1,1067977,https://lenta.ru/news/2021/04/01/trillion/,Байден представил план по спасению экономики н...,1617246403,1617246403,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2021/04/01/06/202...,1617246403,Фото: Reuters Милана Микаилова Президент США Д...,"Президент США отметил, что это крупнейшее влож...",Фото: Reuters Милана Микаилова Президент ... э...
2,2,1067979,https://lenta.ru/news/2021/04/01/business/,Бизнес в России увеличил прибыль,1617248460,1617252223,1,1,0,0,4,[9],https://icdn.lenta.ru/images/2021/04/01/06/202...,1617248460,Фото: Евгений Биятов / РИА Новости Милана Мика...,В январе 2021 года обороты российских предприя...,"Фото: Евгений Биятов / РИА Новости ... больше,..."
3,3,1067984,https://lenta.ru/news/2021/04/01/kredit/,Россиянам назвали способ исправить плохую кред...,1617253029,1617253029,1,1,0,0,4,"[10, 11]",https://icdn.lenta.ru/images/2021/04/01/07/202...,1617253029,Фото: Владимир Трефилов / РИА Новости Илья Соб...,Для этого следует завести новую кредитную карту,Фото: Владимир Трефилов / РИА Новости ... испр...
4,4,1067989,https://lenta.ru/news/2021/04/01/finansist/,Финансист предостерег россиян от покупки подде...,1617255774,1617255775,1,1,0,0,4,[],https://icdn.lenta.ru/images/2021/04/01/08/202...,1617255774,Фото: Globallookpress.com Лика Миргородская Фи...,Он призвал конвертировать валюту безналичным с...,Фото: Globallookpress.com Лика ... россиян от ...


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39948 entries, 0 to 39947
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   39948 non-null  int64 
 1   docid        39948 non-null  int64 
 2   url          39948 non-null  object
 3   title        39948 non-null  object
 4   modified     39948 non-null  int64 
 5   lastmodtime  39948 non-null  int64 
 6   type         39948 non-null  int64 
 7   domain       39948 non-null  int64 
 8   status       39948 non-null  int64 
 9   part         39948 non-null  int64 
 10  bloc         39948 non-null  int64 
 11  tags         39948 non-null  object
 12  image_url    39878 non-null  object
 13  pubdate      39948 non-null  int64 
 14  text         39889 non-null  object
 15  rightcol     39948 non-null  object
 16  snippet      39874 non-null  object
dtypes: int64(10), object(7)
memory usage: 5.2+ MB


In [20]:
df.to_csv('lenta_2021_2024.csv')